# HW Exercise 3

## A Data
The data is obtained from Blooberg, which includes the week prices (09/07/2021,09/14/2021,09/21/2021,09/28/2021) of top 100 current market cap rank constituents of S&P 500 index.

In [1]:
import pandas as pd
import numpy as np

data = pd.read_excel("sp500_top100.xlsx",index_col=0)
data.head()

,AAPL UW Equity,MSFT UW Equity,GOOG UW Equity,GOOGL UW Equity,AMZN UW Equity,FB UW Equity,TSLA UW Equity,BRK/B UN Equity,NVDA UW Equity,JPM UN Equity,...,ADI UW Equity,ZTS UN Equity,COP UN Equity,USB UN Equity,MO UN Equity,GILD UW Equity,TJX UN Equity,PNC UN Equity,ADP UW Equity,LRCX UW Equity
2021-09-07,156.69,300.18,2910.38,2885.61,3509.29,382.18,752.92,278.90,226.62,159.21,...,163.27,207.81,55.98,55.83,50.79,71.19,69.85,192.21,202.23,593.19
2021-09-14,148.12,299.79,2868.12,2850.89,3450.00,376.53,744.49,277.20,222.42,157.07,...,173.47,203.55,56.94,55.91,48.37,70.45,69.48,188.35,199.74,603.50
2021-09-21,143.43,294.80,2792.93,2780.66,3343.63,357.48,739.38,273.18,212.46,152.98,...,170.57,201.01,59.33,57.10,48.03,71.73,69.29,183.97,197.05,586.22
2021-09-28,141.91,283.52,2723.68,2716.60,3315.96,340.65,777.56,276.79,206.99,166.08,...,171.29,192.77,67.80,60.42,48.27,69.97,70.08,197.72,197.28,585.97


## B-C. Signal & Portfolio

B. The signals are generated from log returns weekly. Since there are 4 weeks in total, 3 week of signals are generated and shown below.

C. The portfolio positions for each week are shown above, where figures larger than 0 indicate long position, figures lower than 0 indicate short position. There are 20 stocks to long and 20 stocks to short each week, forming a 20*20 portfolio. 

The current signal weighting portfolio is not perfectly hedged because it did not eliminate market risk or account for volatility and correlations between stocks within the portfolio. 

My suggestions is that 
1) the market risk can be hedged out by directly hedging using SPX ETF using the method from Exercise 2;
2) the weight of stocks can be further optimized by allocating the risk (volatility) of all stocks to the same level, which is called the risk-parity strategy to achieve higher risk-adjusted return and to be more resistent to market downturns. As for correlation, we can look at the covariance matrix of stocks and strategically chose low-correlated assets to optimize low-correlation target.

In [2]:
def CalSignal(data):
    df_return = data.apply(lambda x: np.log(x/x.shift(1)).dropna())
    
    return df_return

def CalWeight(signal, notional):
    def quintile(x):
        if x >= wk.quantile(q=0.8):
            return 1
        elif x <= wk.quantile(q=0.2):
            return -1
        else:
            return 0
    
    pf_all = []
    for i in range(signal.shape[0]):
        wk = signal.iloc[i,:]
        df_signal = wk-wk.median()
        dirc_sig = wk.map(lambda x: quintile(x))
        pf_wk = dirc_sig.copy()
#         print(dirc_sig)
        long_stock = wk[dirc_sig==1]
        long_stock = long_stock / long_stock.sum()
        pf_wk[long_stock.index] = long_stock
        short_stock = wk[dirc_sig==-1]
        short_stock = - short_stock / short_stock.sum()
        pf_wk[short_stock.index] = short_stock
        pf_all.append(pf_wk)
    pf_all = pd.concat(pf_all,axis=1).T
    pf_all_dollar = pf_all * notional/2
    return pf_all, pf_all_dollar

In [3]:
all_return = CalSignal(data)
print("Signals (log return): \n", all_return)

Signals (log return): 
             AAPL UW Equity  MSFT UW Equity  GOOG UW Equity  GOOGL UW Equity  \
2021-09-14       -0.056247       -0.001300       -0.014627        -0.012105   
2021-09-21       -0.032176       -0.016785       -0.026566        -0.024943   
2021-09-28       -0.010654       -0.039014       -0.025107        -0.023307   

            AMZN UW Equity  FB UW Equity  TSLA UW Equity  BRK/B UN Equity  \
2021-09-14       -0.017040     -0.014894       -0.011260        -0.006114   
2021-09-21       -0.031317     -0.051918       -0.006887        -0.014608   
2021-09-28       -0.008310     -0.048224        0.050349         0.013128   

            NVDA UW Equity  JPM UN Equity  ...  ADI UW Equity  ZTS UN Equity  \
2021-09-14       -0.018707      -0.013533  ...       0.060599      -0.020713   
2021-09-21       -0.045814      -0.026384  ...      -0.016859      -0.012557   
2021-09-28       -0.026083       0.082162  ...       0.004212      -0.041857   

            COP UN Equity  US

In [4]:
# Portfolio weight
pf_weight, pf_position = CalWeight(all_return, 2000000)
for i in range(pf_position.shape[0]):
    pf_position_use = pf_position.iloc[i,:]
    pf_position_use = pf_position_use[pf_position_use != 0]
    print("\nPositions of week %s [$]: \n" % (i+1), pf_position_use)


Positions of week 1 [$]: 
 AAPL UW Equity     -58620.668481
MSFT UW Equity      -4044.868782
JNJ UN Equity      -46503.972167
BAC UN Equity      -38014.781886
HD UN Equity        39192.881363
MA UN Equity         9822.593932
PG UN Equity        41349.404268
PYPL UW Equity     -39918.483583
CRM UN Equity      -43493.520179
NFLX UW Equity     -50956.041626
XOM UN Equity        1710.591914
CMCSA UW Equity    -84761.554595
PFE UN Equity      -46500.292708
KO UN Equity         1117.558027
INTC UW Equity      50048.503032
LLY UN Equity      -92813.141728
PEP UW Equity       12458.002561
AVGO UW Equity      32906.114477
WFC UN Equity      119838.404584
MRK UN Equity      -54483.977331
MCD UN Equity       64402.411275
TXN UW Equity       91426.816942
TMUS UW Equity     -43928.125241
PM UN Equity       -41088.443561
CHTR UW Equity     -51991.494881
INTU UW Equity      43000.836539
SCHW UN Equity     -34545.019632
BMY UN Equity      -50441.937253
SBUX UW Equity      81416.401457
UNP UN Equity  

## D. Transaction cost
1. Fill and Opportunity cost: if the trade amount (each stock) is lower than 10000 dollar, a lower fill cost is expected. If the trading amount is higher than 10000 dollar, higher fill cost is expected because of bid-ask spread. 
2. Fee_cost: A fixed percentage cost is expected for each trade. 

The figures are shown in the code.

In [5]:
def TS_Cost(pf_week):
    """Transaction cost function"""
    
    fill_1 = 0.00003 # fill cost + opportunitiy cost - 0.5 bps for lower than $50000
    fill_2 = 0.00005 # fill cost + opportunitiy cost - 1 bps for higher than $50000
    fee_cost = 0.00005 # 0.2 bps since notional for each trade is below 1m
    
    fee_add = pf_week.map(lambda x: abs(x)*(fill_1+fee_cost) if x < 50000 else abs(x)*(fill_2+fee_cost))
    total_fee = fee_add.sum()
    
    return fee_add, total_fee

In [6]:
total_fees = {}
for i in range(pf_position.shape[0]):
    pf_wk = pf_position.iloc[i,:]
    wk_cost, wk_cost_sum = TS_Cost(pf_wk)
    total_fees['week '+str(i+1)] = round(wk_cost_sum,4)
print(total_fees)

{'week 1': 176.3331, 'week 2': 177.1516, 'week 3': 171.2068}


## E Backtest


In [7]:
def Backtest(all_return, pf_position):
    net_returns = []
    for i in range(all_return.shape[0]-1):
        pure_return = all_return.iloc[i+1,:] * pf_position.iloc[i,:]
        wk_cost = TS_Cost(pf_position.iloc[i+1,:])[0]
        net_return = pure_return - wk_cost
        total_return = net_return.sum()
        print("Total return: {:.2f}".format(total_return))
        net_returns.append(net_return)
    net_returns = pd.concat(net_returns, axis=1).T
    
    return net_returns

pf_net_return = Backtest(all_return, pf_position)
pf_net_return.index = ['09140921','09210928']

Total return: 650.12
Total return: 6815.77


In [8]:
pf_total_return = pf_net_return.sum(axis=1).sum()
print(round(pf_total_return,2))

7465.89


The total return for week 09/14/2021 - 09/21/2021 is 650.12.

The total return for week 09/21/2021 - 09/28/2021 is 6815.77.

Total return is 7465.89

In [9]:
# writer = pd.ExcelWriter("wq2155_result_hw3.xlsx")
# all_return.to_excel(writer, sheet_name='Signal')
# pf_position.to_excel(writer, sheet_name='Portfolio Construction')
# pf_net_return.to_excel(writer, sheet_name='Backtest')
# writer.save()